In [1]:
import gc
import numpy as np
import xarray as xr
import rioxarray
import numpy as np

In [2]:
tile = xr.open_zarr("/shared/italy/measurements/V1M0R1/EQUI7_EU500M/E048N006T6/SM-EST-IN_20160101T050131_20171231T171119_VV_SIG0-R-CRRL_E048N006T6_EU500M_V1M0R1__TUWIEN.zarr")
tile

<xarray.Dataset>
Dimensions:                  (time_LAI300: 147, parameter_LAI300: 1, y: 1200,
                              x: 1200, orbit_PLIA-TAG-MEAN: 12,
                              parameter_PLIA-TAG-MEAN: 1,
                              time_SIG0-R-CRRL: 1140, parameter_SIG0-R-CRRL: 1,
                              parameter_swvl1: 1, time_swvl1: 1142)
Coordinates:
  * orbit_PLIA-TAG-MEAN      (orbit_PLIA-TAG-MEAN) <U4 'A015' 'A044' ... 'D168'
    orbit_SIG0-R-CRRL        (time_SIG0-R-CRRL) <U4 dask.array<chunksize=(1140,), meta=np.ndarray>
  * parameter_LAI300         (parameter_LAI300) int64 1
  * parameter_PLIA-TAG-MEAN  (parameter_PLIA-TAG-MEAN) int64 1
  * parameter_SIG0-R-CRRL    (parameter_SIG0-R-CRRL) int64 1
  * parameter_swvl1          (parameter_swvl1) <U5 'swvl1'
  * time_LAI300              (time_LAI300) datetime64[ns] 2015-12-31 ... 2020...
  * time_SIG0-R-CRRL         (time_SIG0-R-CRRL) datetime64[ns] 2016-01-01T05:...
  * time_swvl1               (time_swvl1) datetime64[ns] 2016-01-01 ... 2018-...
  * x                        (x) float64 4.8e+06 4.801e+06 ... 5.399e+06 5.4e+06
  * y                        (y) float64 1.2e+06 1.199e+06 ... 6.002e+05
Data variables:
    LAI300                   (time_LAI300, parameter_LAI300, y, x) float32 dask.array<chunksize=(147, 1, 100, 100), meta=np.ndarray>
    PLIA-TAG-MEAN            (orbit_PLIA-TAG-MEAN, parameter_PLIA-TAG-MEAN, y, x) float32 dask.array<chunksize=(12, 1, 100, 100), meta=np.ndarray>
    SIG0-R-CRRL              (time_SIG0-R-CRRL, parameter_SIG0-R-CRRL, y, x) float32 dask.array<chunksize=(1140, 1, 100, 100), meta=np.ndarray>
    crs                      int64 ...
    spatial_ref              int64 ...
    swvl1                    (time_swvl1, parameter_swvl1, y, x) float32 dask.array<chunksize=(1142, 1, 100, 100), meta=np.ndarray>
Attributes:
    prime_variable:  SIG0-R-CRRL

In [3]:
scores = rioxarray.open_rasterio("/shared/italy/validation/V1M0R1/EQUI7_EU500M/E048N006T6/CORR-SM-swvl1_20170101T000000_20171231T235959_VV_SIG0-R-CRRL_E048N006T6_EU500M_V1M0R1__TUWIEN.tif")
scores

<xarray.DataArray (band: 1, y: 1200, x: 1200)>
[1440000 values with dtype=float64]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 4.8e+06 4.801e+06 4.801e+06 ... 5.399e+06 5.4e+06
  * y            (y) float64 1.2e+06 1.199e+06 1.199e+06 ... 6.008e+05 6.002e+05
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

In [4]:
indices = np.nonzero(~np.isnan(scores[0]))
indices

<xarray.DataArray (y: 2, x: 521470)>
array([[   0,    0,    0, ..., 1199, 1199, 1199],
       [   0,    1,    2, ...,  874,  875,  876]])
Coordinates:
    band         int64 1
  * x            (x) float64 4.8e+06 4.801e+06 4.801e+06 ... 5.399e+06 5.4e+06
  * y            (y) float64 1.2e+06 1.199e+06 1.199e+06 ... 6.008e+05 6.002e+05
    spatial_ref  int64 0

In [5]:
indices.values

array([[   0,    0,    0, ..., 1199, 1199, 1199],
       [   0,    1,    2, ...,  874,  875,  876]])

In [5]:
selection = np.arange(indices.shape[-1])
np.random.shuffle(selection)
indices_sel = indices.values[:, selection[:1000]]

In [ ]:
selected = {}
for k, v in tile.items():
    if {'y', 'x'}.issubset(v.dims):
        selected[k] = v[..., indices_sel[0], indices_sel[1]].load()
        v.close()
    else:
        selected[k] = v.load()
    gc.collect()

tls = xr.Dataset(selected)
tls.attrs = tile.attrs

/home/braml/code/github/use-case-wetland-water-stress/venv/lib/python3.8/site-packages/xarray/core/indexing.py:1380: PerformanceWarning: Slicing with an out-of-order index is generating 77 times more chunks
  value = value[(slice(None),) * axis + (subkey,)]
/home/braml/code/github/use-case-wetland-water-stress/venv/lib/python3.8/site-packages/xarray/core/indexing.py:1380: PerformanceWarning: Slicing with an out-of-order index is generating 76 times more chunks
  value = value[(slice(None),) * axis + (subkey,)]


Frozen({'time_LAI300': (147,), 'parameter_LAI300': (1,), 'y': (1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 2, 1, 4, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [9]:
swvl1 = tile['swvl1'].load()

In [13]:
swvl1

<xarray.DataArray 'swvl1' (time_swvl1: 1142, parameter_swvl1: 1, y: 1200,
                           x: 1200)>
array([[[[0.33699998, 0.3371    , 0.3371    , ...,        nan,
                 nan,        nan],
         [0.3371    , 0.3371    , 0.3372    , ...,        nan,
                 nan,        nan],
         [0.3371    , 0.3371    , 0.3372    , ...,        nan,
                 nan,        nan],
         ...,
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan]]],


       [[[0.33699998, 0.33699998, 0.3371    , ...,        nan,
                 nan,        nan],
         [0.33699998, 0.33699998, 0.3371    , ...,        nan,
                 nan,        nan],
         [0.33699998, 0.3371    , 0.3371    , ...,        nan,
...
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan]]],


       [[[0.3877    , 0.38779998, 0.38779998, ...,        nan,
                 nan,        nan],
         [0.3877    , 0.38779998, 0.38779998, ...,        nan,
                 nan,        nan],
         [0.3877    , 0.38779998, 0.3879    , ...,        nan,
                 nan,        nan],
         ...,
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan]]]], dtype=float32)
Coordinates:
  * parameter_swvl1  (parameter_swvl1) <U5 'swvl1'
  * time_swvl1       (time_swvl1) datetime64[ns] 2016-01-01 ... 2018-01-01
  * x                (x) float64 4.8e+06 4.801e+06 ... 5.399e+06 5.4e+06
  * y                (y) float64 1.2e+06 1.199e+06 ... 6.008e+05 6.002e+05
Attributes:
    long_name:  Volumetric soil water layer 1
    name:       swvl1
    units:      m**3 m**-3

In [11]:
tsl = tile.isel(y=indices_sel[0], x=indices_sel[1])

/home/braml/code/github/use-case-wetland-water-stress/venv/lib/python3.8/site-packages/xarray/core/indexing.py:1380: PerformanceWarning: Slicing with an out-of-order index is generating 76 times more chunks
  value = value[(slice(None),) * axis + (subkey,)]


In [12]:
def norm_chunks(dim, chunk, t):
    if dim in {'y', 'x'}:
        return t.sizes[dim]
    return chunk

tsl = tsl.chunk({d: norm_chunks(d, c, tsl) for d, c in tsl.chunks.items()})

In [13]:
tsl.to_zarr("/tmp/sampled.zarr")


KeyboardInterrupt

